In [1]:
from importlib import reload

import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

sys.path.append(parent_dir)

import src

from src.utils import load_MNIST_data
from src.model import FashionMNISTModel
from src.train import train_model
from src.inference import evaluate_overall, evaluate_classwise, confusion_matrix_plot, plot_roc_curves
reload(src.train)
reload(src.model)
reload(src.utils)
reload(src.inference)
from src.model import FashionMNISTModel

import torch
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR

In [2]:
train_dloader, val_dloader, test_dloader = load_MNIST_data(batch_size = 512, num_workers=4) # try increase batch size to 128

print(len(train_dloader.dataset))
print(len(val_dloader.dataset))
print(len(test_dloader.dataset))

288000
12000
10000


In [ ]:
device = torch.device("mps")

CNN_Model = FashionMNISTModel()

loss_fcn = nn.CrossEntropyLoss()
optimiser = torch.optim.AdamW(params = CNN_Model.parameters(), lr = 0.001, weight_decay=1e-4)

scaler = torch.amp.GradScaler(device)  

epochs = 2

scheduler = OneCycleLR(
    optimiser,
    max_lr=2e-3,
    steps_per_epoch=len(train_dloader),
    epochs=epochs,
    pct_start=0.3,      # portion of cycle to increase LR
    anneal_strategy='cos',  # cosine annealing down
)

for epoch in range(epochs):
    train_loss, val_loss, val_acc = train_model(CNN_Model, optimiser, loss_fcn, train_dloader, val_dloader, epoch, device, scaler, scheduler)


[Epoch  1] Training accuracy: 82.21%, Validation accuracy: 86.61%


Epoch 2:   0%|          | 0/563 [00:00<?, ?it/s]

In [4]:
for epoch in range(8):
    train_loss, val_loss, val_acc = train_model(CNN_Model, optimiser, loss_fcn, train_dloader, val_dloader, epoch, device, scaler)

[Epoch  1] Training accuracy: 91.93%, Validation accuracy: 91.17%


[Epoch  2] Training accuracy: 93.22%, Validation accuracy: 92.20%


[Epoch  3] Training accuracy: 94.02%, Validation accuracy: 90.42%


[Epoch  4] Training accuracy: 95.09%, Validation accuracy: 92.16%


[Epoch  5] Training accuracy: 95.97%, Validation accuracy: 91.54%


[Epoch  6] Training accuracy: 96.76%, Validation accuracy: 90.69%


[Epoch  7] Training accuracy: 97.40%, Validation accuracy: 91.81%


[Epoch  8] Training accuracy: 97.98%, Validation accuracy: 91.57%


In [5]:
test_acc = evaluate_overall(CNN_Model, test_dloader, device)
print(f"Test accuracy: {test_acc:.4f}")

Test accuracy: 0.9202
